In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from datetime import timedelta, date
from sklearn.neural_network import MLPRegressor
import matplotlib as mpl
%matplotlib inline

In [2]:
data = pd.read_csv('/Users/Rohil/Documents/iGEM/yemen/prelim_training_data.csv')

In [3]:
data.date = pd.to_datetime(data.date, format = '%Y-%m-%d')

In [4]:
data = data.sort_values('date').reset_index(drop=True)

In [5]:
data[data.date<'2017-08-15']

,gov_iso,date,weekly_cases,mean_past_week_cases,mean_past_2_week_cases,mean_past_month_cases,neighbor_mean_past_week_cases,neighbor_mean_past_2_week_cases,neighbor_mean_past_month_cases,mean_past_week_rainfall,...,YE-LA,YE-MA,YE-MR,YE-MW,YE-RA,YE-SA,YE-SD,YE-SH,YE-SN,YE-TA
0,YE-AB,2017-05-31,7.853387,0.996581,0.984709,0.984709,0.956582,0.954912,0.954912,2.881060,...,0,0,0,0,0,0,0,0,0,0
1,YE-SN,2017-05-31,11.477125,1.313100,1.320918,1.320918,1.639691,1.646216,1.646216,0.077031,...,0,0,0,0,0,0,0,0,1,0
2,YE-SH,2017-05-31,0.012279,0.006578,0.005756,0.005756,0.931639,0.928049,0.928049,0.087383,...,0,0,0,0,0,0,0,1,0,0
3,YE-BA,2017-05-31,11.652744,2.445220,2.450092,2.450092,0.867570,0.860548,0.860548,0.677152,...,0,0,0,0,0,0,0,0,0,0
4,YE-IB,2017-05-31,8.009573,0.375929,0.380011,0.380011,1.649472,1.606312,1.606312,1.905266,...,0,0,0,0,0,0,0,0,0,0
5,YE-SA,2017-05-31,39.580615,4.643352,4.771159,4.771159,1.313100,1.320918,1.320918,0.000000,...,0,0,0,0,0,1,0,0,0,0
6,YE-DA,2017-05-31,16.109842,2.053912,2.012071,2.012071,0.989144,0.988694,0.988694,2.032561,...,0,0,0,0,0,0,0,0,0,0
7,YE-SD,2017-05-31,0.168074,0.028928,0.025312,0.025312,1.833053,1.837646,1.837646,0.000000,...,0,0,0,0,0,0,1,0,0,0
8,YE-MW,2017-05-31,23.163472,3.000866,3.003987,3.003987,2.130961,2.092793,2.092793,0.155152,...,0,0,0,1,0,0,0,0,0,0
9,YE-TA,2017-05-31,5.678411,0.717478,0.715785,0.715785,1.209102,1.152902,1.152902,6.173968,...,0,0,0,0,0,0,0,0,0,1


In [6]:
X = data.set_index('date').iloc[:,2:].reset_index()
y = data.set_index('date').iloc[:,1].reset_index()

In [7]:
X.head()

,date,mean_past_week_cases,mean_past_2_week_cases,mean_past_month_cases,neighbor_mean_past_week_cases,neighbor_mean_past_2_week_cases,neighbor_mean_past_month_cases,mean_past_week_rainfall,mean_past_2_week_rainfall,mean_past_month_rainfall,...,YE-LA,YE-MA,YE-MR,YE-MW,YE-RA,YE-SA,YE-SD,YE-SH,YE-SN,YE-TA
0,2017-05-31,0.996581,0.984709,0.984709,0.956582,0.954912,0.954912,2.881060,1.473280,1.294694,...,0,0,0,0,0,0,0,0,0,0
1,2017-05-31,1.313100,1.320918,1.320918,1.639691,1.646216,1.646216,0.077031,0.300145,1.656253,...,0,0,0,0,0,0,0,0,1,0
2,2017-05-31,0.006578,0.005756,0.005756,0.931639,0.928049,0.928049,0.087383,0.043691,0.155582,...,0,0,0,0,0,0,0,1,0,0
3,2017-05-31,2.445220,2.450092,2.450092,0.867570,0.860548,0.860548,0.677152,0.360187,0.683651,...,0,0,0,0,0,0,0,0,0,0
4,2017-05-31,0.375929,0.380011,0.380011,1.649472,1.606312,1.606312,1.905266,1.738863,3.047647,...,0,0,0,0,0,0,0,0,0,0


In [8]:
y.tail()

,date,weekly_cases
5392,2018-02-11,0.000000
5393,2018-02-11,1.343834
5394,2018-02-11,0.016594
5395,2018-02-11,0.000000
5396,2018-02-11,0.228348


In [9]:
pd.date_range(start='2017-06-05', end='2018-02-11', freq = 'W-MON')

DatetimeIndex(['2017-06-05', '2017-06-12', '2017-06-19', '2017-06-26',
               '2017-07-03', '2017-07-10', '2017-07-17', '2017-07-24',
               '2017-07-31', '2017-08-07', '2017-08-14', '2017-08-21',
               '2017-08-28', '2017-09-04', '2017-09-11', '2017-09-18',
               '2017-09-25', '2017-10-02', '2017-10-09', '2017-10-16',
               '2017-10-23', '2017-10-30', '2017-11-06', '2017-11-13',
               '2017-11-20', '2017-11-27', '2017-12-04', '2017-12-11',
               '2017-12-18', '2017-12-25', '2018-01-01', '2018-01-08',
               '2018-01-15', '2018-01-22', '2018-01-29', '2018-02-05'],
              dtype='datetime64[ns]', freq='W-MON')

In [10]:
def flatten(lst):
    
    return([val for sublist in lst for val in sublist])

In [96]:
def scale_features(scaler, X, categorical_cols):
    
    X_numerical = X.iloc[:,:(categorical_cols[0])]
    X_categorical = X.iloc[:, categorical_cols[0]:categorical_cols[1]]
        
    X_numerical_scaled = pd.DataFrame(data=scaler.transform(X_numerical), columns = X_numerical.columns)
    
    X_scaled = pd.concat([X_numerical_scaled, X_categorical.reset_index(drop=True)], axis=1)
    
    return (X_scaled)

In [351]:
def train_model(model_type, model_params, X_train, y_train):
    
    if model_type == 'xgboost':
        
        xgb = xgboost.XGBRegressor(**model_params)
        xgb.fit(X_train,y_train)
        
        return (xgb)
    
    if model_type == 'mlp':
        
        mlp = MLPRegressor(hidden_layer_sizes=(6,7,5))
        
        mlp.fit(X_train, y_train)
        
        return (mlp)
    
    if model_type == 'linear_reg':

        regr = LinearRegression()
        
        regr.fit(X_train,y_train)
        
        return (regr)
        

In [200]:
def walk_forward_validation(X, y, model_type, model_params, min_train_date, categorical_cols):
        
    prediction_list = []
    y_total_test =  y[y.date>=min_train_date].drop('date', axis=1)
    
    X_base_numerical = X[(X.date<min_train_date)].drop('date', axis=1).iloc[:,:(categorical_cols[0])]
    scaler = StandardScaler()
    scaler.fit(X_base_numerical)
    
    for date in pd.date_range(start=min_train_date, end='2018-02-11', freq = 'W-MON'):
        
        X_train, X_test = X[(X.date<date)].drop('date', axis=1), X[(X.date>=date) & (X.date < (date + timedelta(7)))].drop('date', axis=1)
        y_train, y_test = y[(y.date<date)].drop('date', axis=1), y[(y.date>=date) & (y.date < (date + timedelta(7)))].drop('date', axis=1)
        #print('train=%d, test=%d' % (X_train.shape[0], X_test.shape[0]))

        X_train, X_test = scale_features(scaler, X_train, categorical_cols), scale_features(scaler, X_test, categorical_cols)
        print('train=%d, test=%d' % (X_train.shape[0], X_test.shape[0]))
        
        model = train_model(model_type, model_params, X_train, y_train)
        
        predictions = model.predict(X_test)
        prediction_list.append(predictions)
        
    return(prediction_list, y_total_test)

In [150]:
def walk_forward_grid_search(X, y, model_params, min_train_date, categorical_cols):
        
    my_cv = [data[data.date<'2017-07-31'].index, ]
    
    for date in pd.date_range(start=min_train_date, end='2018-02-11', freq = '2W-MON'):
        
        train_test_idx_tup = (X[X.date<date].index, X[(X.date>=date) & (X.date < (date + timedelta(14)))].index)
        my_cv.append(train_test_idx_tup)
        
    
    return(my_cv)

In [281]:
def time_series_split_grid_search(X, y, split_date, model_type, param_grid, categorical_cols):
    
    X_train, X_test = X[(X.date<split_date)].drop('date', axis=1), X[(X.date>=split_date)].drop('date', axis=1)
    y_train, y_test = y[(y.date<split_date)].drop('date', axis=1), y[(y.date>=split_date)].drop('date', axis=1)
    
    train_indices, test_indices = y_train.index, y_test.index                                                                                
                                                                                    
    scaler = StandardScaler()
    scaler.fit(X_train.iloc[:,:(categorical_cols[0])])
    
    X_full = scale_features(scaler, X.drop('date', axis=1), categorical_cols)
    y_full = y.drop('date', axis=1)
    
    my_cv = [train_indices, test_indices]
    #my_cv = [data[(data.date<'2017-07-31')].index, data[(data.date>='2017-07-31')&(data.date<'2017-09-25')].index]
    
    print (my_cv)
    
    if model_type == 'xgboost':
        
        xgb = xgboost.XGBRegressor()

        grid_search = GridSearchCV(xgb,
                            param_grid = param_grid,
                            cv = TimeSeriesSplit(n_splits=3).split(X_full),
                            n_jobs = -1,
                            scoring = 'explained_variance',
                            verbose=True)
        
        grid_search.fit(X_full, y_full)
        
    if model_type == 'mlp':
        
        mlp = MLPRegressor(model_params)
    
    return(grid_search.best_score_, grid_search.best_params_)

In [290]:

xgb_params = {'colsample_bytree': 0.5,
  'learning_rate': 0.01,
  'max_depth': 13,
  'min_child_weight': 20,
  'n_estimators': 250,
  'nthread': 4,
  'objective': 'reg:linear',
  'silent': 1,
  'subsample': 0.5}

# {'colsample_bytree': 0.65,
#   'learning_rate': 0.01,
#   'max_depth': 16,
#   'min_child_weight': 30,
#   'n_estimators': 300,
#   'nthread': 4,
#   'objective': 'reg:linear',
#   'silent': 1,
#   'subsample': 0.65}

# {'colsample_bytree': 0.6,
#   'learning_rate': 0.01,
#   'max_depth': 7,
#   'min_child_weight': 15,
#   'n_estimators': 400,
#   'nthread': 4,
#   'objective': 'reg:linear',
#   'silent': 1,
#   'subsample': 0.6}


# {'colsample_bytree': 0.6,
#   'learning_rate': 0.01,
#   'max_depth': 5,
#   'min_child_weight': 2,
#   'n_estimators': 1,
#   'nthread': 4,
#   'objective': 'reg:linear',
#   'silent': 1,
#   'subsample': 0.6}


# {'colsample_bytree': 0.75,
#   'learning_rate': 0.01,
#   'max_depth': 3,
#   'min_child_weight': 12,
#   'n_estimators': 200,
#   'nthread': 4,
#   'objective': 'reg:linear',
#   'silent': 1,
#   'subsample': 0.3}


In [288]:
xgb_param_grid = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [0.01], #so called `eta` value
              'max_depth': [13,15,17],
              'min_child_weight': [15, 20, 25],
              'silent': [1],
              'subsample': [0.5, 0.6],
              'colsample_bytree': [0.5, 0.6],
              'n_estimators': [200, 250, 300,],
}

In [289]:
time_series_split_grid_search(X[X.date<'2017-09-25'], y[y.date<'2017-09-25'], '2017-08-14', 'xgboost', xgb_param_grid, (12,33))

[Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1565, 1566, 1567, 1568, 1569, 1570, 1571, 1572, 1573, 1574],
           dtype='int64', length=1575), Int64Index([1575, 1576, 1577, 1578, 1579, 1580, 1581, 1582, 1583, 1584,
            ...
            2447, 2448, 2449, 2450, 2451, 2452, 2453, 2454, 2455, 2456],
           dtype='int64', length=882)]
Fitting 3 folds for each of 108 candidates, totalling 324 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   51.1s
[Parallel(n_jobs=-1)]: Done 324 out of 324 | elapsed:  1.5min finished


(0.8029103244445318,
 {'colsample_bytree': 0.5,
  'learning_rate': 0.01,
  'max_depth': 13,
  'min_child_weight': 20,
  'n_estimators': 250,
  'nthread': 4,
  'objective': 'reg:linear',
  'silent': 1,
  'subsample': 0.5})

In [352]:
mlp_predictions, y_test = walk_forward_validation(X, y, 'mlp', None, '2017-08-07', (12,33))

train=1428, test=147


C:\Users\Rohil\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Rohil\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


train=1575, test=147
train=1722, test=147
train=1869, test=147
train=2016, test=147
train=2163, test=147
train=2310, test=147
train=2457, test=147
train=2604, test=147
train=2751, test=147
train=2898, test=147
train=3045, test=147
train=3192, test=147
train=3339, test=147
train=3486, test=147
train=3633, test=147
train=3780, test=147
train=3927, test=147
train=4074, test=147
train=4221, test=147
train=4368, test=147
train=4515, test=147
train=4662, test=147
train=4809, test=147
train=4956, test=147
train=5103, test=147
train=5250, test=147


In [339]:
xgboost_predictions, y_test = walk_forward_validation(X, y, 'xgboost', xgb_params, '2017-08-07', (12,33))

train=1428, test=147
train=1575, test=147
train=1722, test=147
train=1869, test=147
train=2016, test=147
train=2163, test=147
train=2310, test=147
train=2457, test=147
train=2604, test=147
train=2751, test=147
train=2898, test=147
train=3045, test=147
train=3192, test=147
train=3339, test=147
train=3486, test=147
train=3633, test=147
train=3780, test=147
train=3927, test=147
train=4074, test=147
train=4221, test=147
train=4368, test=147
train=4515, test=147
train=4662, test=147
train=4809, test=147
train=4956, test=147
train=5103, test=147
train=5250, test=147


In [340]:
linreg_predictions, y_test = walk_forward_validation(X, y, 'linear_reg', None, '2017-08-07', (12,33))

train=1428, test=147
train=1575, test=147
train=1722, test=147
train=1869, test=147
train=2016, test=147
train=2163, test=147
train=2310, test=147
train=2457, test=147
train=2604, test=147
train=2751, test=147
train=2898, test=147
train=3045, test=147
train=3192, test=147
train=3339, test=147
train=3486, test=147
train=3633, test=147
train=3780, test=147
train=3927, test=147
train=4074, test=147
train=4221, test=147
train=4368, test=147
train=4515, test=147
train=4662, test=147
train=4809, test=147
train=4956, test=147
train=5103, test=147
train=5250, test=147


In [353]:
mlp_preds = pd.Series(flatten(mlp_predictions))
xgb_preds = pd.Series(flatten(xgboost_predictions))
lr_preds = pd.Series(flatten(flatten(linreg_predictions)))

In [354]:
pred_dict = {'mlp': mlp_preds, 'xgb': xgb_preds, 'lr' : lr_preds}

In [355]:
for key, value in pred_dict.items():
    
    mean_error = mean_absolute_error(y_test, value)
    explained_variance = explained_variance_score(y_test, value)
    
    print ('%s: mean abs error = %s, explained variance score = %s' % (key, mean_error, explained_variance))

mlp: mean abs error = 2.503118552028095, explained variance score = 0.7682470637749279
xgb: mean abs error = 1.7512052604434403, explained variance score = 0.8498159395217939
lr: mean abs error = 2.6473590182810347, explained variance score = 0.7695573213195307


In [356]:
pred_crosstab_dict = {}
for key, value in pred_dict.items():
    val = value
    val.index = y_test.index
    
    prediction_df = pd.DataFrame(val).merge(pd.DataFrame(data[['gov_iso', 'date']]), left_index = True, right_index = True)
    prediction_df.columns = ['pred', 'gov_iso' ,'date']
        
    pred_crosstab_dict[key] = prediction_df.pivot_table(index = 'date', columns = 'gov_iso', values = 'pred')

In [357]:
y_test_df = y.drop('date', axis=1).merge(pd.DataFrame(data[['gov_iso' ,'date']]), left_index = True, right_index = True)
y_test_crosstab = y_test_df.pivot_table(index = 'date', columns = 'gov_iso', values = 'weekly_cases')

In [358]:
y_test_crosstab.head()

gov_iso,YE-AB,YE-AD,YE-AM,YE-BA,YE-DA,YE-DH,YE-HD-AL,YE-HJ,YE-HU,YE-IB,...,YE-LA,YE-MA,YE-MR,YE-MW,YE-RA,YE-SA,YE-SD,YE-SH,YE-SN,YE-TA
date,,,,,,,,,,,,,,,,,,,,,
2017-05-31,7.853387,13.167803,32.415759,11.652744,16.109842,9.466921,0.0,18.740863,9.986982,8.009573,...,2.625525,2.448999,0.0,23.163472,11.446652,39.580615,0.168074,0.012279,11.477125,5.678411
2017-06-01,8.405887,11.983601,33.929278,12.699393,21.715429,10.586107,0.0,19.486937,11.311516,9.337257,...,3.664075,2.632608,0.0,23.933585,14.303009,42.529852,0.180898,0.039907,12.512657,6.571019
2017-06-02,8.958386,10.799400,35.442796,13.746042,27.321017,11.705293,0.0,20.233010,12.636049,10.664942,...,4.702626,2.816217,0.0,24.703698,17.159365,45.479089,0.193723,0.067536,13.548189,7.463628
2017-06-03,9.510886,9.615199,36.956315,14.792691,32.926605,12.824479,0.0,20.979084,13.960583,11.992626,...,5.741176,2.999826,0.0,25.473811,20.015721,48.428327,0.206548,0.095164,14.583722,8.356236
2017-06-04,9.494190,6.325547,35.491447,13.803081,32.798828,12.447947,0.0,19.014812,13.478453,12.228111,...,6.698759,3.090970,0.0,23.915834,20.636611,46.302799,0.192373,0.092094,13.425218,8.417056


In [359]:
mpl.rcParams.update(mpl.rcParamsDefault)

In [360]:
def set_style(color):
    plt.style.use(['seaborn-' + color, 'seaborn-paper'])

In [361]:
def plot_pred_against_actual(pred_crosstab_dict, test_crosstab):
    
    set_style('white')
    
    fig, ax = plt.subplots(21,1,figsize = (6,15), sharex=True)
    
    cols = test_crosstab.columns
    
    for i in range(0,21):
        
        test_crosstab[cols[i]].plot(kind='line', ax = ax[i], label = 'true_val', legend = True, color = 'red')
        
        ax[i].set_color_cycle(['seagreen', 'blue', 'plum'])
        
        for key, value in pred_crosstab_dict.items():
            
            pred_crosstab_dict[key][cols[i]].plot(kind='line', ax = ax[i], label= key, legend = True)   
          
        ax[i].legend().set_visible(False)
        ax[i].set_ylabel(cols[i])
        ax[i].yaxis.set_label_position('right')
        ax[i].spines['right'].set_visible(False)
        ax[i].spines['top'].set_visible(False)
        ax[i].spines['bottom'].set_visible(True)
        
    ax[10].legend().set_visible(True)
    ax[10].legend(fontsize=10, loc='center left', bbox_to_anchor=(1.05, 0.5))
 
    fig.subplots_adjust(hspace = .2)    
        
    fig.savefig('/Users/Rohil/Documents/iGEM/yemen/plot_xgb_tuned.png', dpi = 500, bbox_inches = 'tight')
    
    plt.close('all')

In [362]:
plot_pred_against_actual(pred_crosstab_dict, y_test_crosstab)

C:\Users\Rohil\Anaconda3\lib\site-packages\matplotlib\cbook.py:136: MatplotlibDeprecationWarning: The set_color_cycle attribute was deprecated in version 1.5. Use set_prop_cycle instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [ ]:
def calculate_validation_by_gov(pred_crosstab_dict, test_crosstab):
        
    cols = test_crosstab.columns
    
    for col in cols:
                
        test = test_crosstab[col]
        
        for key, value in pred_crosstab_dict.items():
            
            pred = pred_crosstab_dict[key][cols[i]]
            
            mean_absolute_error(test, pred)
            
            explained_variance_score(test, pred)

    

In [627]:
y_test.mean()

weekly_cases    7.022306
dtype: float64

In [628]:
y_test.std()

weekly_cases    8.220067
dtype: float64